In [1]:
import mne
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy import signal
from scipy.stats import entropy
mne.utils.use_log_level('error')

In [2]:
#Add new featres in feature list and again in getFeatures method's "flist"
feature_list = ['PSD Mean', 'PSD Median', 'PSD StdDev','PSD CoV','PSD Skew','PSD Kurt', 'PSE']
columns = ['Subject','Test', 'Channel', *feature_list]

In [3]:
#subject = subject number
#testtype = VR or Non-VR
def loadRawData(subject, testtype):
    path = "Preprocessed_Data/Subject-"+str(subject)+"_"+testtype+".fif"
    raw = mne.io.read_raw_fif(path, verbose='error')
    return raw

def getFeatures(raw, time):
    features = pd.DataFrame(columns = feature_list)
    for i in range(time[0],time[1],5): 
        start, stop = raw.time_as_index([i, i+5])
        
        #only looking at CH 4
        picks = mne.pick_types(raw.info, eeg=True, exclude=['CH 1','CH 2','CH 3','CH 5','CH 6','CH 7','CH 8'])
        
        try:    
            d, t = raw[picks[:], start:stop]
            ds = pd.DataFrame(d[0])
            #normalize
            #ds = (ds-ds.mean())/(ds.max()-ds.min())
            
            freq, psd = signal.welch(ds[0], 128, nperseg=256) #nperseg is 2*sf
            
            dp = pd.DataFrame(psd[:41])
            mean = dp.mean()[0]
            median = dp.median()[0]
            std = dp.std()[0]
            mos = mean/std
            skew = dp.skew()[0]
            kurt = dp.kurt()[0]
            pse = entropy(psd)
            
            #add new features here too 
            flist = [mean, median, std, mos, skew, kurt, pse]
            features = features.append(pd.Series(flist, index=feature_list), ignore_index=True)
        except:
            continue
    return features

def getDataFrame(f, data):
    for j in range(len(f)):
        fl = [None]*len(feature_list)
        k=0
        for feature in feature_list:
            fl[k] = f[feature][j]
            k+=1
        data = data.append(pd.Series([i, 'VR', 'CH 4', *fl], index=columns), ignore_index=True)
    return data

def getDiff(df):
    diffDF = df.copy()
    if('Subject' in df.columns):
        diffDF.drop(['Subject'], axis=1, inplace=True)
        diffDF.drop(['Test'], axis=1, inplace=True)
        diffDF.drop(['Channel'], axis=1, inplace=True)
    diffDF = diffDF.diff().dropna()
    return diffDF

In [4]:
#Test 1
#testtype = VR or Non-VR
df1 = pd.DataFrame(columns = columns)
for i in range(1,33):
    data = loadRawData(i,"VR")
    f = getFeatures(raw=data, time=(60,120))
    df1 = getDataFrame(f,df1)

In [5]:
df = df1

In [6]:
#feature_list = ['PSD Mean', 'PSD Median', 'PSD StdDev','PSD CoV','PSD Skew','PSD Kurt', 'PSE', 
#                'Dif1 PSD Mean', 'Dif1 PSD Median', 'Dif1 PSD StdDev','Dif1 PSD CoV','Dif1 PSD Skew','Dif1 PSD Kurt', 'Dif1 PSE',
#                'Dif2 PSD Mean', 'Dif2 PSD Median', 'Dif2 PSD StdDev','Dif2 PSD CoV','Dif2 PSD Skew','Dif2 PSD Kurt', 'Dif2 PSE']
#columns = ['Subject','Test', 'Channel', *feature_list]

#df = pd.concat([df1, df2, df3], axis=1)
#df.columns = columns
#df = df.dropna()

In [7]:
def distanceCalculation(df):
    subs = df['Subject'].unique()    # All subjects
    all_subs= list(itertools.combinations(subs, 2)) # All possible combination for all subjects

    distance_col = ['Subject',*feature_list, 'Type']
    intra_data = pd.DataFrame(columns = distance_col)

    #Intra Distance Computation (Same Person)
    for sub in subs:
        rows = df.loc[df['Subject'] == sub]
        each_comb = list(itertools.combinations(rows.index, 2))
        for i in range(len(each_comb)):
            comb = each_comb[i]
            fdr = absDistance(df, feature_list, comb[0], comb[1])
            intra_data = intra_data.append(pd.Series([sub,*fdr,0], index=distance_col), ignore_index=True)
    
    inter_data = pd.DataFrame(columns = distance_col)
    # Inter Distance Computation (Different Person) 
    all_rows=len(df)
    for sub_pair in all_subs: # Pairs of subjets
        sp1 = df.loc[df['Subject'] == sub_pair[0]].index
        sp2 = df.loc[df['Subject'] == sub_pair[1]].index
        for i in range(len(sp1)):
            for j in range(len(sp2)):
                fdr = absDistance(df, feature_list, sp1[i], sp2[j])
                inter_data = inter_data.append(pd.Series([sub_pair, *fdr, 1], index=distance_col), ignore_index=True)    
    return intra_data, inter_data

def absDistance(df, features, s1, s2):
    r=0
    fdr = [None]*len(features)
    for feature in features:
        f1 = df.iloc[s1][feature] 
        f2 = df.iloc[s2][feature] 
        Inter_dis = np.absolute(f1-f2) # absolute difference
        fdr[r] = Inter_dis
        r+=1
    return fdr

In [8]:
#15 min to run both (6 fetures), 18 min (8 features) - 4/17
intra1, inter1 = distanceCalculation(df)

In [9]:
print("Intra length: "+str(len(intra1)))
print("Inter length: "+str(len(inter1)))

Intra length: 1914
Inter length: 58464


In [10]:
import random
random.seed(1234)
#takes subframe and returns a more managble table for SVM
def get_SVM_Table(intra, inter):
    svmTable = pd.DataFrame()
    rands = random.sample(range(0, len(intra)), 1800)
    for rand in rands:
        svmTable = svmTable.append(intra.iloc[rand],ignore_index=True)
    
    rands = random.sample(range(0, len(inter)), 1800)
    for rand in rands:
        svmTable = svmTable.append(inter.iloc[rand],ignore_index=True)
    return svmTable

In [11]:
svm1 = get_SVM_Table(intra1,inter1)

In [12]:
#all_svm = pd.concat([svm1, svm2], ignore_index=True)

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [14]:
def svmTest(svm):
    X = svm.drop(['Subject','Type'], axis=1)
    y = svm["Type"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    
    model = SVC(C=1.0, kernel = 'rbf', degree=3, gamma='auto')
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    confusionMatrix = confusion_matrix(y_test, prediction)
    print(confusionMatrix)
    print(classification_report(y_test, prediction))
    print("Accuracy: "+str(accuracy_score(y_test, prediction)))
    return confusionMatrix, model

In [15]:
cm, model = svmTest(svm1)

[[402 151]
 [163 364]]
              precision    recall  f1-score   support

         0.0       0.71      0.73      0.72       553
         1.0       0.71      0.69      0.70       527

   micro avg       0.71      0.71      0.71      1080
   macro avg       0.71      0.71      0.71      1080
weighted avg       0.71      0.71      0.71      1080

Accuracy: 0.7092592592592593


In [16]:
TN = cm[0][0]
FN = cm[1][0]
TP = cm[1][1]
FP = cm[0][1]
sums = TN+TP+FN+FP

acc = (TN+TP)/sums

print('False Acceptance: '+str(FP/sums))
print('False Rejection: '+str(FN/sums))
print(acc)

False Acceptance: 0.1398148148148148
False Rejection: 0.15092592592592594
0.7092592592592593
